# Stratify and import GU content 

In [18]:
import pandas as pd
from db.gcp import GCPClient as gcp

In [2]:
urls_and_levels_query="""
    SELECT
      DISTINCT 
      path,
      url,
      section_id,
      web_publication_date
    FROM
      datalake.content_new
    WHERE
      EXTRACT(DATE FROM web_publication_date) >= '2010-01-01' 
      AND word_count>0
      AND body_html <> ''
      AND content_type NOT IN ('Audio','Video','Picture','Crossword')
    """

In [3]:
url_levels=gcp().get_dataframe_from_sql(urls_and_levels_query)

2022-07-27 03:46:57 PM [INFO] - Retrieved 1407608 rows to treat


In [4]:
url_levels.shape

(1407608, 4)

In [12]:
url_levels['year']=url_levels['web_publication_date'].dt.year
url_levels['month']=url_levels['web_publication_date'].dt.month

In [16]:
url_levels=url_levels.groupby(['section_id','year','month'], group_keys=False).apply(lambda x: x.sample(frac=0.2))

In [17]:
url_levels

,path,url,section_id,web_publication_date,year,month
72614,/100-teachers/2021/jan/22/turtles-dont-do-math...,www.theguardian.com/100-teachers/2021/jan/22/t...,100-teachers,2021-01-22 12:27:25+00:00,2021,1
68432,/100-teachers/2021/jan/22/black-and-asian-pupi...,www.theguardian.com/100-teachers/2021/jan/22/b...,100-teachers,2021-01-22 12:29:40+00:00,2021,1
9939,/100-teachers/2021/feb/05/why-i-teach-i-know-t...,www.theguardian.com/100-teachers/2021/feb/05/w...,100-teachers,2021-02-05 15:31:41+00:00,2021,2
60070,/100-teachers/2021/feb/05/every-days-a-school-...,www.theguardian.com/100-teachers/2021/feb/05/e...,100-teachers,2021-02-05 15:26:40+00:00,2021,2
50652,/100-teachers/2021/mar/08/students-respond-to-...,www.theguardian.com/100-teachers/2021/mar/08/s...,100-teachers,2021-03-08 10:26:24+00:00,2021,3
...,...,...,...,...,...,...
65167,/xero-resilient-business/2020/sep/28/how-this-...,www.theguardian.com/xero-resilient-business/20...,xero-resilient-business,2020-09-27 22:50:44+00:00,2020,9
31694,/you-could-be-here/2022/may/11/legendary-scene...,www.theguardian.com/you-could-be-here/2022/may...,you-could-be-here,2022-05-11 09:22:39+00:00,2022,5
48415,/you-could-be-here/2022/may/04/the-envy-of-the...,www.theguardian.com/you-could-be-here/2022/may...,you-could-be-here,2022-05-04 14:27:00+00:00,2022,5
39458,/young-arts-critic-competition/enter,www.theguardian.com/young-arts-critic-competit...,young-arts-critic-competition,2010-05-24 23:01:00+00:00,2010,5


In [20]:
gcp().export_dataframe_to_bigquery_table(url_levels,'temp_analytical','ds_lf_jai_stratified_sampling_urls')

## Import final dataset from GCP and export as CSV

In [48]:
sampled_content_query="""SELECT * FROM temp_analytical.ds_lf_random_sample_20_pcent_gu_content"""
sampled_dataset=gcp().get_dataframe_from_sql(sampled_content_query)

2022-07-28 10:51:54 AM [INFO] - Retrieved 281315 rows to treat


In [39]:
sampled_dataset.shape

(281315, 13)

In [40]:
sampled_dataset[sampled_dataset.isna().any(1)].head()

,path,headline,url,content_type,section_id,pillar_id,web_publication_date,word_count,trail_text,production_office,byline,body_text,body_html
17,/careers/careers-blog/successful-grads-share-t...,Successful grads share their career secrets,www.theguardian.com/careers/careers-blog/succe...,Article,careers,NaN,2010-10-26 14:58:26+00:00,958,<p>Ruth Spellman talks to three young managers...,Uk,Ruth Spellman,October is traditionally the month when organi...,<p>October is traditionally the month when org...
19,/careers/careers-blog/my-attempt-to-break-into...,My attempt to break into TV — part 2,www.theguardian.com/careers/careers-blog/my-at...,Article,careers,NaN,2010-04-28 17:22:50+00:00,708,<p>In the second instalment of his career chan...,Uk,Cain Richards,One of the downsides of leaving a job to pursu...,<p>One of the downsides of leaving a job to pu...
141,/advertising/advertising-display-products-news...,National news,www.theguardian.com/advertising/advertising-di...,Article,advertising,NaN,2010-04-30 14:45:56+00:00,151,<p>The news section of the Guardian provides a...,Uk,NaN,The news section of the Guardian provides auth...,<p>The news section of the Guardian provides a...
172,/theguardian/2010/dec/20/archive-shops-offer-a...,"From the archive, 20 December 1907: Shops offe...",www.theguardian.com/theguardian/2010/dec/20/ar...,Article,theguardian,pillar/news,2010-12-20 17:07:48+00:00,431,<strong>Originally published in the Manchester...,Uk,NaN,Manchester shops have felt during the last few...,<p>Manchester shops have felt during the last ...
181,/careers/in-the-week-08-02-10,In the week 08/02/10,www.theguardian.com/careers/in-the-week-08-02-10,Article,careers,NaN,2010-02-11 14:58:22+00:00,329,<p>A grant from the Rural Development Programm...,Uk,Glen Pawsey,"The Trewithen Dairy at Greymare Farm, Cornwall...","<p>The <a href="" http://www.cornishfarmdairy.c..."


In [41]:
sampled_dataset[sampled_dataset.isna().any(1)].shape

(20143, 13)

In [42]:
sampled_dataset.duplicated().any()

False

In [43]:
sampled_dataset[['url']].duplicated().any()

False

In [56]:
# export dataset per year
for year in set(sampled_dataset['web_publication_date'].dt.year):
    export_name=f'sampled_GU_content_{year}.csv'
    sampled_dataset[sampled_dataset['web_publication_date'].dt.year==year].to_csv(export_name,index=False,encoding="utf-8")